In [93]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


# import tempfile
import matplotlib.pyplot as plt
import pydicom
from pydicom.data import get_testdata_file
import cv2
import os
from skimage import morphology
import math
import numpy as np
from scipy import ndimage
from tqdm import tqdm
import glob as glob
import SimpleITK as sitk
import sys
from pydicom.encaps import encapsulate
from pydicom.uid import JPEG2000
from pydicom.pixel_data_handlers.util import apply_modality_lut
from imagecodecs import jpeg_encode
import imagecodecs
import random
from pydicom.uid import ExplicitVRLittleEndian
from imagecodecs import jpeg_encode
import imagecodecs
import pylibjpeg
import json
import copy
# # CT

# In[ ]:
def sorted_files(folder):
    ins=[]
    ser=[]
    f= sorted(glob.glob((folder + "/**/*.dcm"),recursive = True))

    for name in tqdm(f):
        try:
            ds = pydicom.dcmread(os.path.join(folder,name))
            temp = float(ds.InstanceNumber)
            temp1 = ds.SeriesInstanceUID
            ins.append(int(temp))
            ser.append(temp1)
        except:
            itkimage = sitk.ReadImage(os.path.join(folder,name))
            temp= float(itkimage.GetMetaData('0020|0013')) # instance number 
            temp1= itkimage.GetMetaData('0020|000e') #series instance UID 
            ins.append(int(temp))
            ser.append(temp1)
    series=np.unique(ser)
    files=[[x for sr,_,x in sorted(zip(ser,ins,f)) if sr==s] for s in series]
    return files

def get_scan(path):
    f= sorted_files(path)
    series=[]
    files=[]
    for i in tqdm (range (len(f))):
        img=[]
        file=[]
        for filename in f[i]:
            itkimage = sitk.ReadImage(os.path.join(path,filename))
            numpyImage = sitk.GetArrayFromImage(itkimage)
            img.append(numpyImage[0,:,:])
            file.append(filename)
        series.append(img)
        files.append(file)
    return series,files

def CT_blurred_sag(image,dicom_img,rows,columns):
    imagee = image>-800
#     plt.imshow(imagee,'gray')
#     plt.title("image greater than -400")
#     plt.show()

    skull = image>1200
#     plt.imshow(skull,'gray')
#     plt.title("skull; image greater than 1000")
#     plt.show()

    kernel = np.ones((30, 30), np.uint8)
    imagee = cv2.morphologyEx(imagee.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)    
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,-1, 255,-1 )
#     plt.imshow(gray,'gray')
#     plt.title("contours")
#     plt.show()

    mid = int(image.shape[0]/2)

    gray = np.where((gray==255), 0, 255)
    gray[:,mid:] = 0
    gray1 = np.zeros_like(imagee)
    gray1 = cv2.drawContours(gray1, contours,-1, 255,10)
#     plt.imshow(gray1,'gray')
#     plt.title("contours2")
#     plt.show()
    gray1[:,mid:] = 0
    gray=np.where((gray1==255), gray1, gray)
    gray=np.where((gray==255),1,0)

#     plt.imshow(out,'gray')
#     plt.title("out")
#     plt.show()

    if np.min(image) == np.min(dicom_img) and np.max(image) == np.max(dicom_img):
#         print("equal")
        blurred_img = cv2.GaussianBlur(image, (101, 101), 400)
        out = np.where((gray==1), blurred_img, image)
        out = np.where(skull>0,image,out)
        out = out.astype(np.int16)
#         print(np.min(out),np.max(out))
    else: 
#         print("not equal")
        blurred_img = cv2.GaussianBlur(dicom_img, (101, 101), 50)
        out = np.where((gray==1), blurred_img, dicom_img)
        out = np.where(skull>0,dicom_img,out)
        out = out.astype(np.int16)
#         print(np.min(out),np.max(out))
        
#     for i in range(np.shape(out)[0]):
#         if not np.any(temp[i]):
#             out[i]=np.multiply(out[i],0)
#             out = out.astype(np.int16)
    
    return out
    
def CT_blurred_axial(image, dicom_img, rows, columns):
    imagee = image>-50
#     plt.imshow(imagee,'gray')
#     plt.title("image greater than -50")
#     plt.show()
    
    skull = image>500
#     plt.imshow(skull,'gray')
#     plt.title("skull; image greater than 500")
#     plt.show()
    
    kernel = np.ones((30, 30), np.uint8)
    imagee = cv2.morphologyEx(imagee.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)    
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,-1, 255,-1 )
#     plt.imshow(gray,'gray')
#     plt.title("contours")
#     plt.show()
    gray = np.where((gray==255), 0, 255)
    gray[300:,:] = 0
    gray1 = np.zeros_like(imagee)
    gray1 = cv2.drawContours(gray1, contours,-1, 255,7)
#     plt.imshow(gray1,'gray')
#     plt.title("contours2")
#     plt.show()
    gray1[300:,:] = 0
    gray=np.where((gray1==255), gray1, gray)
    gray=np.where((gray==255),1,0)
    
    if np.min(image) == np.min(dicom_img) and np.max(image) == np.max(dicom_img):
#         print("equal")
        blurred_img = cv2.GaussianBlur(image, (101, 101), 400)
        out = np.where((gray==1), blurred_img, image)
        out = np.where(skull>0,image,out)
        out = out.astype(np.int16)
#         print(np.min(out),np.max(out))
        return out
    else: 
#         print("not equal")
        blurred_img = cv2.GaussianBlur(dicom_img, (101, 101), 50)
        out = np.where((gray==1), blurred_img, dicom_img)
        out = np.where(skull>0,dicom_img,out)
        out = out.astype(np.int16)
#         print(np.min(out),np.max(out))
        return out
        
def CT_blurred_coronal(image, dicom_img, rows, columns):

    imagee = image>-400
#     plt.imshow(imagee,'gray')
#     plt.title("image greater than -400")
#     plt.show()

    skull = image>1000
#     plt.imshow(skull,'gray')
#     plt.title("skull; image greater than 1000")
#     plt.show()

    kernel = np.ones((30, 30), np.uint8)
    imagee = cv2.morphologyEx(imagee.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)    
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,-1, 255,-1 )
#     plt.imshow(gray,'gray')
#     plt.title("contours")
#     plt.show()

    mid = int(image.shape[0]/2)

    gray = np.where((gray==255), 0, 255)
    gray[:200,:] = 0
    gray1 = np.zeros_like(imagee)
    gray1 = cv2.drawContours(gray1, contours,-1, 255,10)
#     plt.imshow(gray1,'gray')
#     plt.title("contours2")
#     plt.show()
    gray1[:200,:] = 0
    gray=np.where((gray1==255), gray1, gray)
    gray=np.where((gray==255),1,0)

    if np.min(image) == np.min(dicom_img) and np.max(image) == np.max(dicom_img):
#         print("equal")
        blurred_img = cv2.GaussianBlur(image, (101, 101), 400)
        out = np.where((gray==1), blurred_img, image)
        out = np.where(skull>0,image,out)
        out = out.astype(np.int16)
#         print(np.min(out),np.max(out))
    else: 
#         print("not equal")
        blurred_img = cv2.GaussianBlur(dicom_img, (101, 101), 50)
        out = np.where((gray==1), blurred_img, dicom_img)
        out = np.where(skull>0,dicom_img,out)
        out = out.astype(np.int16)
#         print(np.min(out),np.max(out))
    
    return out

def predictions_ct(path, output):
    if not os.path.exists(output):
        os.makedirs(output)
    output_list= []
    classUID = []
    annotations=[]
#     start_files = timeit.default_timer()
    scan_series,filenames=get_scan(path)
#     stop_files = timeit.default_timer()
#     print("time for files: ", stop_files - start_files)
    
#     start_ai = timeit.default_timer()
    for j in tqdm(range(len(scan_series))):
        scan, names = scan_series[j],filenames[j]
        scan=np.asarray(scan)
        # print(scan.shape[0])
        # print("no.of files: ",len(names))
        dcmData = pydicom.dcmread(os.path.join(path,names[0]))
        
#         plt.imshow(dcmData.pixel_array,'gray')
#         plt.show()
        
        axial  =  np.array([1., 0., 0., 0., 1., 0.])
#         sagittal = np.array([-1, 0., 0., 0., 0., -1.])
        sagittal = np.array([0, 1., 0., 0., 0., -1.])
        coronal = np.array([1., 0., 0., 0., 0., -1.])
        if (0x0020,0x0037) in dcmData:
            a = (np.around(dcmData.ImageOrientationPatient, 0))
#             print(a)
            if (a == axial).all():
#                 print("in axial")
                for i in range(scan.shape[0]):
                    name = names[i]
                    img = scan[i, :, :]
#                     print("max and min of scan sitk ",np.min(img),np.max(img))
                    dcmData = pydicom.dcmread(os.path.join(path,name)) ##read file to change it's pixel data to anonymized
                    dicom_img = dcmData.pixel_array
#                     print("max and min of input dicom ",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
                    r= dcmData.Rows
                    c=dcmData.Columns
                    out= CT_blurred_axial(img,dicom_img,r,c)
                    out = out.astype(np.int16)
                    dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                    dcmData.PixelData = out.tobytes()
#                     print("max and min of output ",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
                    
#                     plt.imshow(dcmData.pixel_array,'gray')
#                     plt.title("axial blurred")
#                     plt.show()
                    _, tail = os.path.split(name)
                    des_path = os.path.join(output, tail)
                    dcmData.save_as(des_path) ##save file in anonym folder
                    classUID.append(str(dcmData.SOPInstanceUID))
                    output_list.append(des_path)
                    annotations.append(None) 
            elif (a==sagittal).all():
#                 print("in sagittal")
                for i in range(scan.shape[0]):
                    name = names[i]
                    img = scan[i, :, :]
                    dcmData = pydicom.dcmread(os.path.join(path,name)) ##read file to change it's pixel data to anonymized
                    r= dcmData.Rows
                    c=dcmData.Columns
                    dicom_img = dcmData.pixel_array

                    out = CT_blurred_sag(img,dicom_img,r,c)
                        
                    out = out.astype(np.int16)
                    dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                    dcmData.PixelData = out.tobytes()
                    
#                     plt.imshow(dcmData.pixel_array,'gray')
#                     plt.title("sagittal blurred")
#                     plt.show()
                    _, tail = os.path.split(name)
                    des_path = os.path.join(output, tail)
                    dcmData.save_as(des_path) ##save file in anonym folder
                    classUID.append(str(dcmData.SOPInstanceUID))
                    output_list.append(des_path)
                    annotations.append(None) 
            elif (a==coronal).all():
#                 print("in coronal")
                for i in range(scan.shape[0]):
                    name = names[i]
                    img = scan[i, :, :]
                    dcmData = pydicom.dcmread(os.path.join(path,name))
                    dicom_img = dcmData.pixel_array
#                     print("max and min of input dicom ",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
                    r= dcmData.Rows
                    c=dcmData.Columns
                    out= CT_blurred_coronal(img,dicom_img,r,c)
                    out = out.astype(np.int16)
                    dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                    dcmData.PixelData = out.tobytes()
#                     print("max and min of output ",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
                    
#                     plt.imshow(dcmData.pixel_array,'gray')
#                     plt.title("coronal blurred")
#                     plt.show()
                    _, tail = os.path.split(name)
                    des_path = os.path.join(output, tail)
                    dcmData.save_as(des_path) ##save file in anonym folder
                    classUID.append(str(dcmData.SOPInstanceUID))
                    output_list.append(des_path)
                    annotations.append(None) 
            else:
                for i in range(scan.shape[0]):
                    name = names[i]
                    img = scan[i, :, :]
                    dcmData = pydicom.dcmread(os.path.join(path,name)) 
                    _, tail = os.path.split(name)
                    des_path = os.path.join(output, tail)
                    dcmData.save_as(des_path) 
                    classUID.append(str(dcmData.SOPInstanceUID))
                    output_list.append(des_path)
                    annotations.append(None) 
        else:
#             print("save as it is")
#             print(j)
            for i in range(scan.shape[0]):
#                 print(j)
                name = names[i]
                img = scan[i, :, :]
                dcmData = pydicom.dcmread(os.path.join(path,name)) 
                _, tail = os.path.split(name)
                des_path = os.path.join(output, tail)
                dcmData.save_as(des_path) 
                classUID.append(str(dcmData.SOPInstanceUID))
                output_list.append(des_path)
                annotations.append(None) 

    mimeType = "application/dicom"
    mimeType = []
    for i in range(len(output_list)):
        mimeType.append("application/dicom")
    image=[]
    print("in AI",len(classUID))
    for i in range(len(classUID)):
        image.append(None)
    recommendation_string = {"finding": "dummy finding","conclusion":"dummy conclusion","recommendation":"dummy recommendation"} 
    all_result={"output_list":output_list, "classUID":classUID, "mimeType":mimeType, "recommendation_string": recommendation_string,"annotations":annotations,"image":image}
    with open(path+"/values.json", "w") as outfile:
        json.dump(all_result, outfile)
#     print(recommendation_string)
    return all_result

# # CBCT 

# In[ ]:
def dcm_image(filename): 
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
    ins = int(itkimage.GetMetaData('0020|0013'))#float(itkimage.GetMetaData('0020|0013'))
    return numpyImage, ins

def get_full_scan(folder_path):
    files_List = glob.glob(folder_path + '/**/*.dcm', recursive=True)
    try:
        ds = pydicom.dcmread(files_List[0])
        rows = ds.Rows
        cols = ds.Columns
        mn = 1000
        mx = 0
        for file in tqdm(files_List):
            ds = pydicom.dcmread(file)
            mn = np.min([mn, int(ds.InstanceNumber)])
            mx = np.max([mx, int(ds.InstanceNumber)])        
    except:
        itkimage = sitk.ReadImage(files_List[0])
        rows = int(itkimage.GetMetaData('0028|0010'))
        cols = int(itkimage.GetMetaData('0028|0011'))
        mn = 1000
        mx = 0
        for file in tqdm(files_List):
            itkimage = sitk.ReadImage(file)
            mn = np.min([mn, int(itkimage.GetMetaData('0020|0013'))])
            mx = np.max([mx, int(itkimage.GetMetaData('0020|0013'))])
            
    full_scan = np.ndarray(shape=(mx - mn + 1, rows, cols), dtype=float, order='F')
    new_list = np.ndarray(shape=(mx - mn + 1), dtype=object)
    
    for file in tqdm(files_List):
        img, n = dcm_image(file)
        n = int(n)
        full_scan[n - mn, :, :] = img[0, :, :]
        new_list[n-mn] = file
        
    return full_scan,new_list

def get_normalized(scan,mn,mx):
    scann = copy.copy(scan)
    try:
        mn = max(mn,np.amin(scann))
        mx = min(mx,np.amax(scann))
    except:
        print("in except")
        scann2 = copy.copy(scan)
        scann2 = scann2[:scann2.shape[0]-1,:,:]
        mn = max(mn,np.amin(scann2))
        mx = min(mx,np.amax(scann2))
    np.clip(scann, mn, mx, out=scann)
    d = mx - mn
    scann = (scann-mn)/d
    return scann

def dynamic_windowing(scan):
    counts,bins,bars = plt.hist(scan.flatten())
    plt.close()
    if bins[-1]>3200:
        a  = 2
#         print("Type 2")
        scan = get_normalized(scan, -800, 2500)
    else:
        scan = scan
        a = 1
    return scan,a

#         counts = sorted(counts)
# #         c1 = counts[-1]/counts[-2]

#         if counts[0]<7000:
        
#             print('Type 1')
#             a=1
#             scan = get_normalized(scan,-250,1800)
#         else:
#             a=3
#             print('Type 3')
#             scan = get_normalized(scan,-250,3000)
#     return scan,a

def blur_1(image, prev = 0, s = 0):
    gray = np.zeros((image.shape[0], image.shape[1]) , dtype=int)
    for i in range(gray.shape[0]):
        for j in range(gray.shape[1]):
            if image[i,j] > -300 and image[i,j] < 500:
                gray[i,j] = 1
#         plt.imshow(gray , cmap = 'gray')
#         plt.title("org gray")
#         plt.show()
#         print(np.min(gray), np.max(gray))


    ker1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,1))
    ker2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    gray = cv2.morphologyEx(np.float32(gray), cv2.MORPH_CLOSE, ker1)
    gray = cv2.morphologyEx(np.float32(gray), cv2.MORPH_CLOSE, ker2)
#     plt.imshow(gray , cmap = 'gray')
#     plt.title("gray after closing")
#     plt.show()
    gray = gray>0
    gray = morphology.remove_small_objects(gray, min_size=200)
#     plt.imshow(gray , cmap = 'gray')
#     plt.title("after removing small objects")
#     plt.show()
    gray = gray.astype(np.uint8)
    kernel = np.ones((30, 30), np.uint8)
    gray = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
    gray = gray.astype(np.uint8)
#     plt.imshow(gray , cmap = 'gray')
#     plt.title("closing")
#     plt.show()
    contours, hierarchy = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    gray = cv2.drawContours(gray, contours,-1, 255, 10)
#     plt.imshow(gray , cmap = 'gray')
#     plt.title("contours")
#     plt.show()
    gray[300:,:] = 0
#     plt.imshow(gray, cmap ='gray')
#     plt.show()
    seg = gray
    seg = np.where((seg==255),seg,0)
    z=0
    if not s==0:
        diff = np.subtract(seg,prev)
        diff = diff>1
        xx = int((diff.shape[1])/2)
        diff[:,0:(xx-100)] =0
        diff[:,(xx+100):] = 0
        diff = morphology.remove_small_objects(diff, min_size=500)
        if np.any(diff):
            for l in range(diff.shape[1]):
                for j in range(diff.shape[0]):
                    if diff[j,l]==True:
                        if np.any(prev[(j-10):(j+10),l]):
                            diff[j,l] = False
            seg = np.where((diff==True),0,seg)
            z=1

    blurred_img = cv2.GaussianBlur(image, (101, 101), 400)
    out = np.where((seg==255), blurred_img, image)
#     plt.imshow(out,'gray')
#     plt.show()
    out = out.astype(np.int16)
#         plt.imshow(out,'gray')
#         plt.show()
    if z==0:
        return out , seg
    else:
        return out, prev

    
def blur_2(img, ds,low, high, prev = 0, s=0):
    # print("low n high: ",low, high)
    dcm = img*np.max(ds.pixel_array) #*2048#*np.max(scan)
#     plt.imshow(dcm,'gray')
#     plt.show()
    # print(np.min(dcm),np.max(dcm))
    gray = np.zeros((dcm.shape[0], dcm.shape[1]) , dtype=int)
    for i in range(gray.shape[0]):
        for j in range(gray.shape[1]):
            if dcm[i,j] > low and dcm[i,j] < high:
                gray[i,j] = 1
#     plt.imshow(gray , cmap = 'gray')
#     plt.title("org gray")
#     plt.show()
    #         print(np.min(gray), np.max(gray))

    ker1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,1))
    ker2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    gray = cv2.morphologyEx(np.float32(gray), cv2.MORPH_CLOSE, ker1)
    gray = cv2.morphologyEx(np.float32(gray), cv2.MORPH_CLOSE, ker2)
    #     plt.imshow(gray , cmap = 'gray')
    #     plt.title("gray after closing")
    #     plt.show()
    gray = gray>0
    gray = morphology.remove_small_objects(gray, min_size=200)
    #     plt.imshow(gray , cmap = 'gray')
    #     plt.title("after removing small objects")
    #     plt.show()
    gray = gray.astype(np.uint8)
    kernel = np.ones((30, 30), np.uint8)
    gray = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
    gray = gray.astype(np.uint8)
    #     plt.imshow(gray , cmap = 'gray')
    #     plt.title("closing")
    #     plt.show()
    contours, hierarchy = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    gray = cv2.drawContours(gray, contours,-1, 255, 10)
    #     plt.imshow(gray , cmap = 'gray')
    #     plt.title("contours")
    #     plt.show()
    gray[400:,:] = 0
#     plt.imshow(gray, cmap ='gray')
#     plt.show()

    seg = gray
    seg = np.where((seg==255),seg,0)
    z=0
    if not s==0:
        diff = np.subtract(seg,prev)
        diff = diff>1
        xx = int((diff.shape[1])/2)
        diff[:,0:(xx-100)] =0
        diff[:,(xx+100):] = 0
        diff = morphology.remove_small_objects(diff, min_size=500)
        if np.any(diff):
            for l in range(diff.shape[1]):
                for j in range(diff.shape[0]):
                    if diff[j,l]==True:
                        if np.any(prev[(j-10):(j+10),l]):
                            diff[j,l] = False
            seg = np.where((diff==True),0,seg)
            z=1

    blurred_img = cv2.GaussianBlur(ds.pixel_array, (101, 101), 800)
    
    out = np.where((seg==255), blurred_img, ds.pixel_array)
#     plt.imshow(out,'gray')

    out = out.astype(np.int16)

#     print(np.min(ds.pixel_array),np.max(ds.pixel_array))
#     ds.PixelData = out.tobytes()

#     plt.imshow(ds.pixel_array,'gray')
#     plt.title("bytes")
#     plt.show()

#     print(np.min(ds.pixel_array),np.max(ds.pixel_array))

    if z==0:
        return out , seg
    else:
        return out, prev   
    

# from skimage.metrics import structural_similarity

# def signaltonoise(a, axis=0, ddof=0):
#     a = np.asanyarray(a)
#     m = a.mean(axis)
#     sd = a.std(axis=axis, ddof=ddof)
#     return np.where(sd == 0, 0, m/sd)


def prediction_CBCT_1(scann, names, output_folder):
    output_list= []
    classUID = []
    annotations=[]

    iscan = copy.copy(scann)
    oscan = copy.copy(scann)    
    
    for i in tqdm(range(scann.shape[0])):
        name = names[i]
        img = scann[i, :, :]

        dcmData = pydicom.dcmread(name)
        dicom_img = dcmData.pixel_array #for calculating parameters
        iscan[i] = dicom_img

        if i == 0:
            out, prev_img = blur_1(img)
        else:
            out, prev_img = blur_1(img, prev_img, s=1)
#             print(np.min(out),np.max(out))    
#             plt.imshow(out,'gray')
#             plt.title("after ct_blur")
#             plt.show()

        out = out.astype(np.int16)
        oscan[i] = out
#             plt.imshow(out,'gray')
#             plt.title("after int16")
#             plt.show()
#             print("after blue and np16: ", np.min(out),np.max(out))

#             dcmData = pydicom.dcmread(name)
        outbytes = out.tobytes()

#             print("bytes min and maximum",np.min(outbytes.pixel_array),np.max(outbytes.pixel_array))
        dcmData.PixelData = outbytes

        # print("pixel data min and maximum",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
        # plt.imshow(dcmData.pixel_array,'gray')
        # plt.title("bytes")
        # plt.show()

        _, tail = os.path.split(name)
        des_path = os.path.join(output_folder, tail)
        dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
        dcmData.save_as(des_path)

        classUID.append(str(dcmData.SOPInstanceUID))
        output_list.append(des_path)
        annotations.append(None)    
    
    # isnr = signaltonoise(iscan, axis=None, ddof=0)
    # osnr = signaltonoise(oscan,axis=None, ddof=0)
    # (ss, dd) = structural_similarity(iscan, oscan, full=True)

    # print(isnr, osnr, ss)
    
    return output_list, classUID, annotations

def prediction_CBCT_2(scann, names, output_folder):
    
    output_list= []
    classUID = []
    annotations=[]    
    
    iscan = copy.copy(scann)
#     oscan = copy.copy(scann)
    
    for i in tqdm(range(scann.shape[0])):
        name = names[i]
        img = scann[i, :, :]
        low = 350
        high = 2000
        dcmData = pydicom.dcmread(name)
        if dcmData.file_meta.TransferSyntaxUID == "1.2.840.10008.1.2.4.70":
            dcmData.decompress('pylibjpeg')
            low = 250
            high = 3000
            
        dicom_img = dcmData.pixel_array
        iscan[i] = dicom_img
        
        if i == 0:
            out, prev_img = blur_2(img, dcmData, low, high)
        else:
            out, prev_img = blur_2(img, dcmData, low, high, prev_img, s=1)
#             print(np.min(out),np.max(out))    
#             plt.imshow(out,'gray')
#             plt.title("after ct_blur")
#             plt.show()

        out = out.astype(np.int16)
#         oscan[i] = out
#             plt.imshow(out,'gray')
#             plt.title("after int16")
#             plt.show()
#             print("after blue and np16: ", np.min(out),np.max(out))

#             dcmData = pydicom.dcmread(name)
        outbytes = out.tobytes()

#             print("bytes min and maximum",np.min(outbytes.pixel_array),np.max(outbytes.pixel_array))
        dcmData.PixelData = outbytes

        # print("pixel data min and maximum",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
        # plt.imshow(dcmData.pixel_array,'gray')
        # plt.title("bytes")
        # plt.show()

        _, tail = os.path.split(name)
        des_path = os.path.join(output_folder, tail)
        dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
        dcmData.save_as(des_path)
        
#         test_ds = pydicom.dcmread(des_path)
#         test_s = test_ds.pixel_array
#         plt.imshow(test_s,'gray')
#         plt.title("read output")
#         plt.show()
        
        classUID.append(str(dcmData.SOPInstanceUID))
        output_list.append(des_path)
        annotations.append(None)
    
    # isnr = signaltonoise(iscan, axis=None, ddof=0)
    # osnr = signaltonoise(oscan,axis=None, ddof=0)
    # (ss, dd) = structural_similarity(iscan, oscan, full=True)

    # print(isnr, osnr, ss)
    
    return output_list, classUID, annotations

def predictions_cbct(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    files_list = sorted(glob.glob((input_folder+ "/**/*.dcm"),recursive = True))
    
    initialdata = pydicom.dcmread(files_list[0])
    mode = str(initialdata.Modality)
    # start_AI = timeit.default_timer()
    if (mode=='CT') or (mode=='ct'):
        i = 0
        # start_files = timeit.default_timer()
        scan, names = get_full_scan(input_folder)
        # stop_files = timeit.default_timer()
        # print("time for reading files: ", (stop_files - start_files))
        
        inscan = copy.copy(scan)
        scann,a = dynamic_windowing(inscan)
        
        if a == 1:
            # print("Type 1, org alg")
            output_list, classUID, annotations = prediction_CBCT_1(scan, names, output_folder)
        elif a == 2:
            # print("Type 2, improv alg")
            output_list, classUID, annotations = prediction_CBCT_2(scann, names, output_folder)
        

#             i = i + 1
    # stop_ai = timeit.default_timer()
    # print("Time taken by AI: ", stop_ai - start_AI)
    

#     mimeType = "application/dicom"
    mimeType = []
    for i in range(len(output_list)):
        mimeType.append("application/dicom")
    image=[]
    print("in AI",len(classUID))
    for i in range(len(classUID)):
        image.append(None)
    recommendation_string = {"finding": "dummy finding","conclusion":"dummy conclusion","recommendation":"dummy recommendation"} 
    all_result={"output_list":output_list, "classUID":classUID, "mimeType":mimeType,"annotations":annotations, "recommendation_string": recommendation_string,"image":image}
    with open(input_folder+"/values.json", "w") as outfile:
        json.dump(all_result, outfile)
    # print(recommendation_string)
        

    return all_result

## MRI
def dynamic_windowing_mri(scan, scan_seriesi):
    counts,bins,bars = plt.hist(scan.flatten())
    
#     print(bins[-1])
    plt.close()
    if bins[-1]>3200:
        a  = 2
#         print("Type 2")
        scan = get_normalized(scan, -800, 2500)
    elif bins[-1]>350:
        a = 3
#         print("Type 3")
        scan = get_normalized(scan, 5, 100)
    else:
        scan = get_normalized(scan, np.min(scan), np.max(scan))
        a = 1
    return scan,a

def MRI_blurred_sag(image):

    rows=image.shape[1]
    columns=image.shape[0]
    head = image>100
    skull = image>500
    ROI=np.logical_xor(head,skull)
    kernel = np.ones((30, 30), np.uint8)
    imagee = cv2.morphologyEx(ROI.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,0, 255,1 )
    #plt.imshow(gray,'gray')
    #plt.show()
    temp=gray.copy()
    temp=np.where((temp==255),1,0)
    variable=int(temp.shape[1]/2)
    temp[:,variable:] = 0
    #plt.imshow(temp,'gray')
    #plt.show()
    
    r=[[random.randint(-250,500) for i in range(rows)] for j in range(columns)]
    r=np.asarray(r)
    blurred_img= np.multiply(temp,r)
    out = np.where((temp==1), blurred_img, image)

    return out

def MRI_blurred_axial(image):
    rows=image.shape[1]
    columns=image.shape[0]
    #print(rows,columns)
    #print(image.shape)
    head = image>100
    skull = image>500
    ROI=np.logical_xor(head,skull)
    kernel = np.ones((30, 30), np.uint8)
    imagee = cv2.morphologyEx(ROI.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,0, 255,1 )
    #plt.imshow(gray,'gray')
    #plt.show()
    temp=gray.copy()
    temp=np.where((temp==255),1,0)
    variable=int((temp.shape[0])/2)
    temp[variable:,:] = 0
    #plt.imshow(temp,'gray')
    #plt.show()

    r=[[random.randint(-250,500) for i in range(rows)] for j in range(columns)]
    r=np.asarray(r)
    #print(r.shape)
    blurred_img= np.multiply(temp,r)
    out = np.where((temp==1), blurred_img, image)

    return out

def MRI_blurred_coronal(image):

    rows=image.shape[1]
    columns=image.shape[0]
    #print(rows,columns)
    #print(image.shape)
    head = image>100
    skull = image>500
    ROI=np.logical_xor(head,skull)
    kernel = np.ones((30, 30), np.uint8)
    imagee = cv2.morphologyEx(ROI.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,0, 255,1 )
    #plt.imshow(gray,'gray')
    #plt.show()
    temp=gray.copy()
    temp=np.where((temp==255),1,0)
    #plt.imshow(temp,'gray')
    #plt.show()
   
    r=[[random.randint(-250,500) for i in range(rows)] for j in range(columns)]
    r=np.asarray(r)
    
    blurred_img= np.multiply(temp,r)
    out = np.where((temp==1), blurred_img, image)

    return out

def predictions_mri_head(path, output_folder):
    print("in head")
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    output_list= []
    classUID = []
    annotations=[]        
    scan_series,filenames = get_scan(path)

    for j in tqdm(range(len(scan_series))):
        scan, names = scan_series[j],filenames[j]
        scan=np.asarray(scan)
        dcmData = pydicom.dcmread(os.path.join(path,names[0]))
        if (dcmData.PhotometricInterpretation == "RGB" ):
#             print("rgb")
            for i in range(scan.shape[0]):
                name = names[i]
                img = scan[i, :, :]
                dcmData = pydicom.dcmread(os.path.join(path,name)) 
                _, tail = os.path.split(name)
                des_path = os.path.join(output_folder, tail)
                dcmData.save_as(des_path) 
                classUID.append(str(dcmData.SOPInstanceUID))
                output_list.append(des_path) 
                annotations.append(None)
        elif (dcmData.PhotometricInterpretation == "rgb"):
#             print("rgb")
            for i in range(scan.shape[0]):
                name = names[i]
                img = scan[i, :, :]
                dcmData = pydicom.dcmread(os.path.join(path,name)) 
                _, tail = os.path.split(name)
                des_path = os.path.join(output_folder, tail)
                dcmData.save_as(des_path) 
                classUID.append(str(dcmData.SOPInstanceUID))
                output_list.append(des_path) 
                annotations.append(None)
        else:
#             print("mono")
            axial  =  np.array([1., 0., 0., 0., 1., 0.])
            #sagittal= np.array([-1., 0., 0., 0., 0., -1.])
            sagittal= np.array([0., 1., 0., 0., 0., -1.])
            coronal= np.array([1., 0., 0., 0., 0., -1.])
            if (0x0020,0x0037) in dcmData:
                a = (np.around(dcmData.ImageOrientationPatient, 0))
    #             print("a",a)
    #             print("scan shape",scan.shape[0])
                image= scan#[20,:,:]
                try:
                    histogram, bin_edges = np.histogram(image, bins=256)
                    check=bin_edges[-1]
                except:
    #                 print("here in except for ", j)
                    image = []
                    for i in range(len(scan_seriesi[j])-1):
                        image.append(np.asarray(scan_seriesi[j][i]))
                    histogram, bin_edges = np.histogram(image, bins=256)
                    check=bin_edges[-1] 

                if ((a == axial).all()) and check>1000 :
    #                 print("axial")
    #                 print(j)
                    #continue
                    for i in range(scan.shape[0]):
                        name = names[i]
                        img = scan[i, :, :]
                        dcmData = pydicom.dcmread(os.path.join(path,name)) ##read file to change it's pixel data to anonymized
#                         c= dcmData.Rows
#                         r=dcmData.Columns
                        dicom_img = dcmData.pixel_array
                        out= MRI_blurred_axial(img)
                        #out = out.astype(np.int16)
                        #print(dicom_img.dtype)
                        out1 = out.astype(dicom_img.dtype)
                        #print(out.shape)
                        dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                        dcmData.PixelData = out1.tobytes()
                        '''
                        plt.imshow(dcmData.pixel_array,'gray')
                        plt.title("axial blurred")
                        plt.show()

                        plt.imshow(out, 'gray')
                        plt.title("axial blurred OUT")
                        plt.show()
                        '''
                        _, tail = os.path.split(name)
                        des_path = os.path.join(output_folder, tail)
                        dcmData.save_as(des_path) ##save file in anonym folder


                        classUID.append(str(dcmData.SOPInstanceUID))
                        output_list.append(des_path)
                        annotations.append(None)

                elif ((a==sagittal).all()) and check>1000:
    #                     print(j)
    #                     print("sag")
                       # continue
                        for i in range(scan.shape[0]):
                            name = names[i]
                            img = scan[i, :, :]
                            dcmData = pydicom.dcmread(os.path.join(path,name)) ##read file to change it's pixel data to anonymized
                            #print(dcmData.file_meta.TransferSyntaxUID)
                            dicom_img = dcmData.pixel_array
#                             r= dcmData.Rows
#                             c=dcmData.Columns
#                             if i == 0:
#                                 out = img
#                             else:
                            out = MRI_blurred_sag(img)
                            #out = out.astype(np.int16)
                            out = out.astype(dicom_img.dtype)
                            #plt.imshow(out,'gray')
                            #plt.show()

                            dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                            #dcmData.pixel_array = out
                            dcmData.PixelData = out.tobytes()
                            #dcmData.PixelData = out.tostring()
                            #dcmData.BitsAllocated = 
                            #dcmData.PixelRepresentation = 1
                            '''
                            plt.imshow(dcmData.pixel_array,'gray')
                            plt.title("sagittal blurred")
                            plt.show()

                            plt.imshow(out, 'gray')
                            plt.title("sagittal blurred OUT")
                            plt.show()
                            '''
                            _, tail = os.path.split(name)
                            des_path = os.path.join(output_folder, tail)
                            dcmData.save_as(des_path) ##save file in anonym folder

                            classUID.append(str(dcmData.SOPInstanceUID))
                            output_list.append(des_path)
                            annotations.append(None)
                elif ((a==coronal).all()) and (scan.shape[0]>100) and  check>1000:
    #                     print(j)
    #                     scan.shape[0]
    #                     print("coronal")
                        #continue

                        for i in range(scan.shape[0]):
    #                     for i in range(50): #distorting the boundary for only the front of the coronal scan i.e the face part
                            if i<50:
                                name = names[i]
                                img = scan[i, :, :]
                                dcmData = pydicom.dcmread(os.path.join(path,name)) ##read file to change it's pixel data to anonymized
                                #print(dcmData.file_meta.TransferSyntaxUID)
                                dicom_img = dcmData.pixel_array
#                                 r= dcmData.Rows
#                                 c=dcmData.Columns
#                                 if i == 0:
#                                     out = img
#                                 else:
                                out = MRI_blurred_coronal(img)
                                #out = out.astype(np.int16)
                                out = out.astype(dicom_img.dtype)
                                #plt.imshow(out,'gray')
                                #plt.show()

                                dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                                #dcmData.file_meta.TransferSyntaxUID= 'Explicit VR Big Endian'
                                #dcmData.pixel_array = out
                                dcmData.PixelData = out.tobytes()
                                #dcmData.PixelData = out.tostring()

                                '''
                                plt.imshow(dcmData.pixel_array,'gray')
                                plt.title("coronal blurred")
                                plt.show()

                                plt.imshow(out, 'gray')
                                plt.title("coronal blurred OUT")
                                plt.show()
                                ''' 
                                _, tail = os.path.split(name)
                                des_path = os.path.join(output_folder, tail)
                                dcmData.save_as(des_path) ##save file in anonym folder

                                classUID.append(str(dcmData.SOPInstanceUID))
                                output_list.append(des_path)
                                annotations.append(None)
                            else:
                                name = names[i]
#                                 img = scan[i, :, :]
                                dcmData = pydicom.dcmread(os.path.join(path,name)) ##read file to change it's pixel data to anonymized
                                #print(dcmData.file_meta.TransferSyntaxUID)

                                dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                                #dcmData.file_meta.TransferSyntaxUID= 'Explicit VR Big Endian'
                                #dcmData.pixel_array = out
                                #dcmData.PixelData = out.tostring()

                                '''
                                plt.imshow(dcmData.pixel_array,'gray')
                                plt.title("coronal blurred")
                                plt.show()

                                plt.imshow(out, 'gray')
                                plt.title("coronal blurred OUT")
                                plt.show()
                                ''' 
                                _, tail = os.path.split(name)
                                des_path = os.path.join(output_folder, tail)
                                dcmData.save_as(des_path) ##save file in anonym folder

                                classUID.append(str(dcmData.SOPInstanceUID))
                                output_list.append(des_path)
                                annotations.append(None)
                else:
                        #print("ccc")
                        #continue
                        for i in range(scan.shape[0]):
                            #print(scan.shape[0],i)
                            name = names[i]
                            #print("name",name)
#                             img = scan[i, :, :]
                            dcmData = pydicom.dcmread(os.path.join(path,name))
                            _, tail = os.path.split(name)
                            des_path = os.path.join(output_folder, tail)
                            dcmData.save_as(des_path) ##save file in anonym folder
                            classUID.append(str(dcmData.SOPInstanceUID))
                            output_list.append(des_path)
                            annotations.append(None)
            else:
                for i in range(scan.shape[0]):
                    name = names[i]
#                     img = scan[i, :, :]
                    dcmData = pydicom.dcmread(os.path.join(path,name)) 
                    _, tail = os.path.split(name)
                    des_path = os.path.join(output_folder, tail)
                    dcmData.save_as(des_path) 
                    classUID.append(str(dcmData.SOPInstanceUID))
                    output_list.append(des_path)
                    annotations.append(None)

    mimeType = "application/dicom"
    mimeType = []
    for i in range(len(output_list)):
        mimeType.append("application/dicom")
    image=[]
    print("in AI",len(classUID))
    for i in range(len(classUID)):
        image.append(None)
    recommendation_string = {"finding": "dummy finding","conclusion":"dummy conclusion","recommendation":"dummy recommendation"} 
    all_result={"output_list":output_list, "classUID":classUID, "mimeType":mimeType, "recommendation_string": recommendation_string,"annotations":annotations,"image":image}
    with open(path+"/values.json", "w") as outfile:
        json.dump(all_result, outfile)
#     print(recommendation_string)
    return all_result

def MRI_brain_blurred_axial(image, scan_img, low=50, high=500):

    rows=image.shape[1]
    columns=image.shape[0]
    #print(rows,columns)
    #print(image.shape)
    head = image > low
    skull = image > high
    ROI=np.logical_xor(head,skull)
    kernel = np.ones((30, 30), np.uint8)
    
    imagee = cv2.morphologyEx(ROI.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,0, 255, 2)
#     plt.imshow(gray,'gray')
#     plt.show()
    temp=gray.copy()
    temp=np.where((temp==255),1,0)
    variable=int((temp.shape[0])/2)
    temp[variable:,:] = 0
    #plt.imshow(temp,'gray')
    #plt.show()
    
    r=[[random.randint(-250,500) for i in range(rows)] for j in range(columns)]
    r=np.asarray(r)
    #print(r.shape)
    blurred_img= np.multiply(temp,r)
    out = np.where((temp==1), blurred_img, scan_img)
    
    #plt.imshow(out,'gray')
    #plt.show()
    return out

def MRI_brain_blurred_sagittal(image, scan_img, low=50, high=500):

    rows=image.shape[1]
    columns=image.shape[0]
    #print(rows,columns)
    #print(image.shape)
    head = image > low
    skull = image > high
    ROI=np.logical_xor(head,skull)
    kernel = np.ones((30, 30), np.uint8)
    
    imagee = cv2.morphologyEx(ROI.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,0, 255, 2)
#     plt.imshow(gray,'gray')
#     plt.show()
    temp=gray.copy()
    temp=np.where((temp==255),1,0)
    variable=int((temp.shape[0])/2)
    temp[:,variable:] = 0
    #plt.imshow(temp,'gray')
    #plt.show()
    
    r=[[random.randint(-250,500) for i in range(rows)] for j in range(columns)]
    r=np.asarray(r)
    #print(r.shape)
    blurred_img= np.multiply(temp,r)
    out = np.where((temp==1), blurred_img, scan_img)
    
    #plt.imshow(out,'gray')
    #plt.show()
    return out

def MRI_brain_blurred_coronal(image, scan_img, low=50, high=500):

    rows=image.shape[1]
    columns=image.shape[0]
    #print(rows,columns)
    #print(image.shape)
    head = image > low
    skull = image > high
    ROI=np.logical_xor(head,skull)
    kernel = np.ones((30, 30), np.uint8)
    
    imagee = cv2.morphologyEx(ROI.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,0, 255, 2)
#     plt.imshow(gray,'gray')
#     plt.show()
    temp=gray.copy()
    temp=np.where((temp==255),1,0)
#     variable=int((temp.shape[0])/2)
#     temp[variable:,:] = 0
    #plt.imshow(temp,'gray')
    #plt.show()
    
    r=[[random.randint(-250,500) for i in range(rows)] for j in range(columns)]
    r=np.asarray(r)
    #print(r.shape)
    blurred_img= np.multiply(temp,r)
    out = np.where((temp==1), blurred_img, scan_img)
    
    #plt.imshow(out,'gray')
    #plt.show()
    return out

def predictions_mri_brain(path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    output_list= []
    classUID = []
    annotations=[]   
    
    scan_series,filenames = get_scan(path)

    for j in tqdm(range(len(scan_series))):
        scan, names = scan_series[j],filenames[j]
        scan = np.asarray(scan)
        if scan[0].shape != scan[-1].shape:
            
            name = names[-1]
            img = scan[-1]
            dcmData = pydicom.dcmread(os.path.join(path,name)) 
            _, tail = os.path.split(name)
            des_path = os.path.join(output_folder, tail)
            dcmData.save_as(des_path) 
            classUID.append(str(dcmData.SOPInstanceUID))
            output_list.append(des_path) 
            annotations.append(None)   
            scan = []
            for i in range(len(scan_series[j])-1):
                scan.append(np.asarray(scan_series[j][i]))
            scan = np.asarray(scan)
#         print("SHAPE: ", scan.shape)
#         print("reading ",j)
        dcmData = pydicom.dcmread(os.path.join(path,names[0]))
#         print("read", j)
        if 'rgb' in str(dcmData.PhotometricInterpretation) or 'RGB' in str(dcmData.PhotometricInterpretation):
#             print("rgb")
            for i in range(scan.shape[0]):
                name = names[i]
                img = scan[i, :, :]
                dcmData = pydicom.dcmread(os.path.join(path,name)) 
                _, tail = os.path.split(name)
                des_path = os.path.join(output_folder, tail)
                dcmData.save_as(des_path) 
                classUID.append(str(dcmData.SOPInstanceUID))
                output_list.append(des_path) 
                annotations.append(None)

        else:
            wind_scan, t = dynamic_windowing_mri(scan, scan_series[j])
#             print("axial")
            axial  =  np.array([1., 0., 0., 0., 1., 0.])
            #sagittal= np.array([-1., 0., 0., 0., 0., -1.])
            sagittal= np.array([0., 1., 0., 0., 0., -1.])
            coronal= np.array([1., 0., 0., 0., 0., -1.])
            if (0x0020,0x0037) in dcmData:
#                 print("in dcmdata orientation")
                a = (np.around(dcmData.ImageOrientationPatient, 0))
    #             print("a",a)
    #             print("scan shape",scan.shape[0])
    #             image= scan#[20,:,:]
    #             histogram, bin_edges = np.histogram(image, bins=256)
    #             check=bin_edges[-1]

                if ((a == axial).all()): #and check > 350 :
#                     print("axial")
    #                 print(j)
                    #continue
                    for i in range(scan.shape[0]):
                        name = names[i]
                        wind_img = wind_scan[i, :, :]
                        scan_img = scan[i, :, :]
#                         print("reading ",i)
                        dcmData = pydicom.dcmread(os.path.join(path,name)) ##read file to change it's pixel data to anonymized
                        c= dcmData.Rows
                        r=dcmData.Columns
                        dicom_img = dcmData.pixel_array
#                         print("read ",i)
                        out= MRI_brain_blurred_axial(wind_img*np.max(scan_img),scan_img, 50, 500)
                        #out = out.astype(np.int16)
                        #print(dicom_img.dtype)
                        out1 = out.astype(dicom_img.dtype)
                        #print(out.shape)
                        dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                        dcmData.PixelData = out1.tobytes()
                        '''
                        plt.imshow(dcmData.pixel_array,'gray')
                        plt.title("axial blurred")
                        plt.show()

                        plt.imshow(out, 'gray')
                        plt.title("axial blurred OUT")
                        plt.show()
                        '''
                        _, tail = os.path.split(name)
                        des_path = os.path.join(output_folder, tail)
                        dcmData.save_as(des_path) ##save file in anonym folder


                        classUID.append(str(dcmData.SOPInstanceUID))
                        output_list.append(des_path)
                        annotations.append(None)

                elif ((a==sagittal).all()): #and check>350:
    #                     print(j)
    #                     print("sag")
                       # continue
                        for i in range(scan.shape[0]):
                            name = names[i]
                            wind_img = wind_scan[i, :, :]
                            scan_img = scan[i, :, :]
                            dcmData = pydicom.dcmread(os.path.join(path,name)) ##read file to change it's pixel data to anonymized
                            c= dcmData.Rows
                            r=dcmData.Columns
                            dicom_img = dcmData.pixel_array
                            out= MRI_brain_blurred_sagittal(wind_img*np.max(scan_img),scan_img, 100, 1000)
    #                         if i == 0:
    #                             out = img
    #                         else:
    #                             out = MRI_blurred_sag(img,dicom_img,r,c)
                            #out = out.astype(np.int16)
                            out = out.astype(dicom_img.dtype)
                            #plt.imshow(out,'gray')
                            #plt.show()

                            dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                            #dcmData.pixel_array = out
                            dcmData.PixelData = out.tobytes()
                            #dcmData.PixelData = out.tostring()
                            #dcmData.BitsAllocated = 
                            #dcmData.PixelRepresentation = 1
                            '''
                            plt.imshow(dcmData.pixel_array,'gray')
                            plt.title("sagittal blurred")
                            plt.show()

                            plt.imshow(out, 'gray')
                            plt.title("sagittal blurred OUT")
                            plt.show()
                            '''
                            _, tail = os.path.split(name)
                            des_path = os.path.join(output_folder, tail)
                            dcmData.save_as(des_path) ##save file in anonym folder

                            classUID.append(str(dcmData.SOPInstanceUID))
                            output_list.append(des_path)
                            annotations.append(None)
                elif ((a==coronal).all()): #and (scan.shape[0]>100) and  check>350:
    #                     print(j)
    #                     scan.shape[0]
    #                     print("coronal")
                        #continue

                        for i in range(scan.shape[0]):
    #                     for i in range(50): #distorting the boundary for only the front of the coronal scan i.e the face part
                            if i<50:
                                name = names[i]
                                wind_img = wind_scan[i, :, :]
                                scan_img = scan[i, :, :]
                                dcmData = pydicom.dcmread(os.path.join(path,name)) ##read file to change it's pixel data to anonymized
                                c= dcmData.Rows
                                r=dcmData.Columns
                                dicom_img = dcmData.pixel_array
                                out= MRI_brain_blurred_coronal(wind_img*np.max(scan_img),scan_img, 100, 1000)
    #                             if i == 0:
    #                                 out = img
    #                             else:
    #                                 out = MRI_blurred_coronal(img,dicom_img,r,c)
                                #out = out.astype(np.int16)
                                out = out.astype(dicom_img.dtype)
                                #plt.imshow(out,'gray')
                                #plt.show()

                                dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                                #dcmData.file_meta.TransferSyntaxUID= 'Explicit VR Big Endian'
                                #dcmData.pixel_array = out
                                dcmData.PixelData = out.tobytes()
                                #dcmData.PixelData = out.tostring()

                                '''
                                plt.imshow(dcmData.pixel_array,'gray')
                                plt.title("coronal blurred")
                                plt.show()

                                plt.imshow(out, 'gray')
                                plt.title("coronal blurred OUT")
                                plt.show()
                                ''' 
                                _, tail = os.path.split(name)
                                des_path = os.path.join(output_folder, tail)
                                dcmData.save_as(des_path) ##save file in anonym folder

                                classUID.append(str(dcmData.SOPInstanceUID))
                                output_list.append(des_path)
                                annotations.append(None)
                            else:
                                name = names[i]
                                img = scan[i, :, :]
                                dcmData = pydicom.dcmread(os.path.join(path,name)) ##read file to change it's pixel data to anonymized
                                #print(dcmData.file_meta.TransferSyntaxUID)

                                dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                                #dcmData.file_meta.TransferSyntaxUID= 'Explicit VR Big Endian'
                                #dcmData.pixel_array = out
                                #dcmData.PixelData = out.tostring()

                                '''
                                plt.imshow(dcmData.pixel_array,'gray')
                                plt.title("coronal blurred")
                                plt.show()

                                plt.imshow(out, 'gray')
                                plt.title("coronal blurred OUT")
                                plt.show()
                                ''' 
                                _, tail = os.path.split(name)
                                des_path = os.path.join(output_folder, tail)
                                dcmData.save_as(des_path) ##save file in anonym folder

                                classUID.append(str(dcmData.SOPInstanceUID))
                                output_list.append(des_path)
                                annotations.append(None)
                        else:
                            #print("ccc")
                            #continue
                            for i in range(scan.shape[0]):
                                #print(scan.shape[0],i)
                                name = names[i]
                                #print("name",name)
                                img = scan[i, :, :]
                                dcmData = pydicom.dcmread(os.path.join(path,name))
                                _, tail = os.path.split(name)
                                des_path = os.path.join(output_folder, tail)
                                dcmData.save_as(des_path) ##save file in anonym folder
                                classUID.append(str(dcmData.SOPInstanceUID))
                                output_list.append(des_path)
                                annotations.append(None)
                else:
                    for i in range(scan.shape[0]):
                        name = names[i]
                        img = scan[i, :, :]
                        dcmData = pydicom.dcmread(os.path.join(path,name)) 
                        _, tail = os.path.split(name)
                        des_path = os.path.join(output_folder, tail)
                        dcmData.save_as(des_path) 
                        classUID.append(str(dcmData.SOPInstanceUID))
                        output_list.append(des_path)
                        annotations.append(None)
            else:
                for i in range(scan.shape[0]):
                    name = names[i]
                    img = scan[i, :, :]
                    dcmData = pydicom.dcmread(os.path.join(path,name)) 
                    _, tail = os.path.split(name)
                    des_path = os.path.join(output_folder, tail)
                    dcmData.save_as(des_path) 
                    classUID.append(str(dcmData.SOPInstanceUID))
                    output_list.append(des_path)
                    annotations.append(None)

    mimeType = "application/dicom"
    mimeType = []
    for i in range(len(output_list)):
        mimeType.append("application/dicom")
    image=[]
    print("in AI",len(classUID))
    for i in range(len(classUID)):
        image.append(None)
    recommendation_string = {"finding": "dummy finding","conclusion":"dummy conclusion","recommendation":"dummy recommendation"} 
    all_result={"output_list":output_list, "classUID":classUID, "mimeType":mimeType, "recommendation_string": recommendation_string,"annotations":annotations,"image":image}
    with open(path+"/values.json", "w") as outfile:
        json.dump(all_result, outfile)
    print(recommendation_string)
    return all_result


# # Main

# In[ ]:


def predictions(input_folder, output_folder, additional_parameters = None):
    files_list = sorted(glob.glob((input_folder+ "/**/*.dcm"),recursive = True))
    initialdata = pydicom.dcmread(files_list[0])
    try:
        if 'ct' in str(initialdata.Modality) or 'CT' in str(initialdata.Modality):
            if (0x0008,0x1030) in initialdata:
                if 'cbct' in str(initialdata.StudyDescription) or 'CBCT' in str(initialdata.StudyDescription):
                    all_result = predictions_cbct(input_folder, output_folder)
#                 elif 'CBCT' in str(initialdata.StudyDescription):
#                     all_result = predictions_cbct(input_folder, output_folder)
                else:
                    flag = 'cbct'
                    ###insert check for number of series from metadata
                    for file in files_list:
                        d = pydicom.dcmread(file)
                        if str(initialdata.SeriesNumber) != d.SeriesNumber:
                            flag = 'ct'
                            break
                    if flag == 'ct':
                        all_result = predictions_ct(input_folder, output_folder)
                    else:
                        all_result = predictions_cbct(input_folder, output_folder)
            else:
                if initialdata.SeriesNumber == None:
                    flag = 'cbct'
                    all_result = predictions_cbct(input_folder, output_folder)
                else:
                    flag = 'ct'
                    all_result = predictions_ct(input_folder, output_folder)
        elif 'MR' in str(initialdata.Modality) or 'mr' in str(initialdata.Modality):
#             print("mr")
            if 'Head' in str(initialdata.StudyDescription) or 'head' in str(initialdata.StudyDescription):
                all_result = predictions_mri_head(input_folder, output_folder)
            elif 'Brain' in str(initialdata.StudyDescription) or 'brain' in str(initialdata.StudyDecription):
        #                 print("brainn")
                all_result = predictions_mri_brain(input_folder, output_folder)

    except Exception as e:
        print("erroror ", e)
        output_files=[]
        mimeType=[]
        classUID=[]
        annotations=[]
        recommendation_string={"status":'Error Occurred',"message":e}
        image=[]
        all_result={"output_files":output_files,"classUID":classUID,"mimeType":mimeType,"annotations":annotations,"recommendation_string":recommendation_string,"image":image}
        with open(input_folder + "/values.json","w") as outfile:
            json.dump(all_result,outfile,skipkeys=True)
    return all_result

def main(input_path,out_path): 
    
    with open(input_path+'additional_paras.json', 'r') as openfile:
        additional_parameters = json.load(openfile)
    
    if all (key in additional_parameters for key in ("add_params")):
        additional_parameters=additional_parameters["add_params"]
        
    predictions(input_path,out_path, additional_parameters)

if __name__ == '__main__':
    main(sys.argv[1], sys.argv[2])

In [91]:
ipath = r"/home/azka/HPACS_v4/Face_Anonymization/input_files/63930f2aefb98316924fe4ff/" #20392703
opath = r"/home/minha/FARZEEN/face ann/face mri/output_/63930f2aefb98316924fe4ff/"
main(ipath, opath)

 31%|█████████████▊                              | 5/16 [00:03<00:04,  2.46it/s]WARNING: In /tmp/SimpleITK-build/ITK/Modules/IO/GDCM/src/itkGDCMImageIO.cxx, line 358
GDCMImageIO (0x55c8581d78a0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c8581d78a0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c8581d78a0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c8581d78a0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c8581d78a0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c8581d78a0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related 

GDCMImageIO (0x55c8581d78a0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c8581d78a0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c8581d78a0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c8581d78a0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c8581d78a0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c8581d78a0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c8581d78a0): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMIm

yo check  0


  6%|██▊                                         | 1/16 [01:12<18:07, 72.52s/it]

yo check  1


...

 75%|████████████████████████████████▎          | 12/16 [01:50<00:10,  2.57s/it]

yo check  12


 81%|██████████████████████████████████▉        | 13/16 [01:52<00:07,  2.49s/it]/tmp/ipykernel_35964/623244490.py:1224: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  scan = np.asarray(scan)
100%|███████████████████████████████████████████| 16/16 [02:13<00:00,  8.31s/it]


in AI 847


PermissionError: [Errno 13] Permission denied: '/home/azka/HPACS_v4/Face_Anonymization/input_files/63930f2aefb98316924fe4ff//values.json'

In [60]:
scan_series, filenames = get_scan(ipath)

 31%|█████████████▊                              | 5/16 [00:03<00:04,  2.48it/s]WARNING: In /tmp/SimpleITK-build/ITK/Modules/IO/GDCM/src/itkGDCMImageIO.cxx, line 358
GDCMImageIO (0x55c87db81a90): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c87db81a90): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c87db81a90): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c87db81a90): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c87db81a90): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c87db81a90): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related 

GDCMImageIO (0x55c87db81a90): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c87db81a90): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c87db81a90): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c87db81a90): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c87db81a90): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c87db81a90): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMImageIO (0x55c87db81a90): Converting from MONOCHROME1 to MONOCHROME2 may impact the meaning of DICOM attributes related to pixel values.

GDCMIm

In [49]:
len(scan_series)

16

In [57]:
scan13 = np.asarray(scan_series[13])

/tmp/ipykernel_35964/698309283.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  scan13 = np.asarray(scan_series[13])


In [75]:
ss = dynamic_windowing_mri(scan13, scan_series[13])

in except


IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [52]:
len(scan_series[13])

20

In [53]:
for i in range(len(scan_series[13])):
    s = np.asarray(scan_series[13][i])
    print(s.shape)

(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(768, 768)
(484, 484)


In [ ]:
try: 
    scan13 = scan